In [1]:
import numpy as np
import sympy
from sympy import prime
from sympy.ntheory.modular import crt
import random
import time
import binascii
import string

In [2]:
def cantor_pair(k1, k2, safe=True):
    z = int(0.5 * (k1 + k2) * (k1 + k2 + 1) + k2)
    if safe and (k1, k2) != cantor_unpair(z):
        raise ValueError("{} and {} cannot be paired".format(k1, k2))
    return z

def cantor_unpair(z):
    w = np.floor((np.sqrt(8 * z + 1) - 1) / 2)
    t = (w**2 + w) / 2
    y = int(z - t)
    x = int(w - y)
    # assert z != pair(x, y, safe=False):
    return (x, y)

def padding(plain_text,padding_character,block_size):
    plain_text += padding_character*(block_size-len(plain_text))
    return plain_text

def chunkstring(string, length):
    return list(string[0+i:length+i] for i in range(0, len(string), length))

def reduced_ascii_chunks(ascii_chunks):
    cantor_reduced_list = []
    for i in range(0,len(ascii_chunks)-1,2):
        cantor_reduced_list.append(cantor_pair(ascii_chunks[i],ascii_chunks[i+1]))
    return cantor_reduced_list

def generate_string(l):
    m = ''.join([random.choice(string.ascii_letters) for n in range(l)])
    return m

In [3]:
message = []
for i in range(4,257):
    message.append(generate_string(i))
print(message)

['cebK', 'TCGsq', 'aezVhd', 'qtBDpCZ', 'NpYkFiFJ', 'uNQeThgJK', 'lYdAANFqgQ', 'YWwoFWgtXAY', 'MofoZRRvJSiK', 'vLALNRYslIEVo', 'ZTiebZnORlAoAN', 'tMmeKBziCvEqVZp', 'jRsMCUfQCbCdSoAj', 'NucFrhUJmLUPhiHkn', 'yQyEhMWvKKNLqqVjFE', 'cAkKztmZJJPuQLRoRpO', 'HmdqBbsWuYIHtbFIrGQy', 'bsOfJRDERZVpruJJGtSwg', 'XSIWLtfEXVxzeBzTdcFARx', 'hRlpfpvDaxDtvIScFtLTkSf', 'jklItRXYXcvIJGXFhLSTwBqR', 'PdUQJqZjeusXRVHSwfPDrgfiZ', 'KqxHHXyvTpPPHYtXFFvHrTPBRT', 'wRuWLqRNqJcRmOWmvdlGPBZECKm', 'CniqrUrHkWSdgtSsXQVkAJZmtwsF', 'ACTBLwHsUSXVrtsBduzjEhPZKamNd', 'bkOyRADCRtvvjfgMFVnWRIgKIcJSsI', 'XxniQqGazhWLjjxSObVGbhFMrDjtjBK', 'hXrXkvfsYyWZuokYGoopPCBihMbgRfRf', 'rdPKgNMeGqxMGFwPESfxIKRpcYPoxkOCX', 'pdOqkveehQFFzTAZjTlfNAKtRrhTCcAdeT', 'YyBkUtmCFomchZTIiPGnVKIRXrcgOLoHeuL', 'YkEZaYcuzchMycKWtRqZzJbXnOBwuphfjyJk', 'YPrcbAWLYzGVFIyLFaPjopHbNOoveAgBfDwrg', 'mWYALNKfQiSTcZYQuCnPxRtHNgXYcGRxOrzxIT', 'UZWVshfWVxhFhCAaLhYFqeJqGhfEBspCsNcIgoo', '...']


In [4]:
data_point = []

In [5]:
for msg in message:
    hex_data   = binascii.hexlify(msg.encode())
    plain_text = str(int(hex_data, 16))
    n = len(plain_text)
    padding_character = '>'
    for block_size in range(4,257):
        M = []
        for i in range(1,block_size//2+1):
            M.append(sympy.ntheory.generate.nextprime(220,ith=i))

        random.shuffle(M)

    #     print('Private key generated : ',end='')
    #     print(M)

        t1 = time.time()

        plain_text_chunks = []
        if n < block_size:
            plain_text_chunks.append(padding(plain_text,padding_character,block_size))
        else:
            chunks = chunkstring(plain_text,block_size)
            for i in range(len(chunks)):
                plain_text_chunks.append(padding(chunks[i],padding_character,block_size))
        # print(plain_text_chunks)

        ascii_chunks = []
        for chunk in plain_text_chunks:
            t = []
            for i in chunk:
                if i.isdigit():
                    t.append(int(i))
                else:
                    t.append(ord(i)-(ord(padding_character)-10))
            ascii_chunks.append(t)
        # print(ascii_chunks)

        cantor_reduced_list = []
        for chunk in ascii_chunks:
            cantor_reduced_list.append(reduced_ascii_chunks(chunk))
        # print(cantor_reduced_list)

        # Applying the Chinese Remainder Theorem to get X
        cipher_text = []
        for chunk in cantor_reduced_list:
            x = crt(M,chunk)
            cipher_text.append(round(x[0]))
    #     print('Encrypted Cipher Text : ',cipher_text)


        # ---------------------------------DECRYPTION-----------------------------------
        # private_key - known
        # cipher_text - known
        # block_size  - known


        dec_M = M

        dec_cantor_reduced_list = []
        for x in cipher_text:
            t = []
            for mi in dec_M:
                t.append(x%mi)
            dec_cantor_reduced_list.append(t)
        # print(dec_cantor_reduced_list)

        decrypted_ascii_list = []
        for chunk in dec_cantor_reduced_list:
            t = []
            for i in range(len(chunk)):
                x,y = cantor_unpair(chunk[i])
                t.append(x)
                t.append(y)
            decrypted_ascii_list.append(t)

        decrypted_text_list = []
        for chunk in decrypted_ascii_list:
            for i in chunk:
                if i!=10:
                    decrypted_text_list.append(i)
        
        decrypted_message = ''.join(map(lambda x:str(x),decrypted_text_list))
        decrypted_message = int(decrypted_message)
        print('{'+str(block_size)+','+str(len(msg))+','+str(time.time()-t1)+'},')


{4,4,0.00013065338134765625},
{5,4,7.224082946777344e-05},
{6,4,0.00012755393981933594},
{7,4,8.821487426757812e-05},
{8,4,0.00010967254638671875},
{9,4,0.00010728836059570312},
{10,4,7.414817810058594e-05},
{11,4,7.104873657226562e-05},
{12,4,9.965896606445312e-05},
{13,4,8.535385131835938e-05},
{14,4,0.0001049041748046875},
{15,4,0.0001049041748046875},
{16,4,9.942054748535156e-05},
{17,4,0.00010228157043457031},
{18,4,0.0001342296600341797},
{19,4,0.0001163482666015625},
{20,4,0.0001270771026611328},
{21,4,0.00012135505676269531},
{22,4,0.00015163421630859375},
{23,4,0.0001513957977294922},
{24,4,0.00033736228942871094},
{25,4,0.0001723766326904297},
{26,4,0.0001914501190185547},
{27,4,0.00018548965454101562},
{28,4,0.00019669532775878906},
{29,4,0.00017452239990234375},
{30,4,0.0001766681671142578},
{31,4,0.00018215179443359375},
{32,4,0.0001938343048095703},
{33,4,0.00018858909606933594},
{34,4,0.00020623207092285156},
{35,4,0.0002579689025878906},
{36,4,0.00022292137145996094},
{

{159,5,0.0009443759918212891},
{160,5,0.000911712646484375},
{161,5,0.0009300708770751953},
{162,5,0.0012090206146240234},
{163,5,0.0011429786682128906},
{164,5,0.0014412403106689453},
{165,5,0.0015213489532470703},
{166,5,0.0010554790496826172},
{167,5,0.0009572505950927734},
{168,5,0.0009341239929199219},
{169,5,0.0009613037109375},
{170,5,0.000978708267211914},
{171,5,0.0009410381317138672},
{172,5,0.00098419189453125},
{173,5,0.0009942054748535156},
{174,5,0.0009763240814208984},
{175,5,0.0009958744049072266},
{176,5,0.000985860824584961},
{177,5,0.0010898113250732422},
{178,5,0.0010199546813964844},
{179,5,0.0010192394256591797},
{180,5,0.001054525375366211},
{181,5,0.0010478496551513672},
{182,5,0.00104522705078125},
{183,5,0.0010561943054199219},
{184,5,0.001604318618774414},
{185,5,0.001073598861694336},
{186,5,0.0015134811401367188},
{187,5,0.001275777816772461},
{188,5,0.002249002456665039},
{189,5,0.0015635490417480469},
{190,5,0.0016973018646240234},
{191,5,0.00273489952087

{210,6,0.0014014244079589844},
{211,6,0.0013480186462402344},
{212,6,0.0013606548309326172},
{213,6,0.0015041828155517578},
{214,6,0.0015811920166015625},
{215,6,0.0014362335205078125},
{216,6,0.0014216899871826172},
{217,6,0.0013723373413085938},
{218,6,0.0013890266418457031},
{219,6,0.0014181137084960938},
{220,6,0.0014281272888183594},
{221,6,0.0014200210571289062},
{222,6,0.001435995101928711},
{223,6,0.0014445781707763672},
{224,6,0.0014679431915283203},
{225,6,0.0013616085052490234},
{226,6,0.0013511180877685547},
{227,6,0.0013899803161621094},
{228,6,0.0014667510986328125},
{229,6,0.0014424324035644531},
{230,6,0.0014309883117675781},
{231,6,0.0014858245849609375},
{232,6,0.0014524459838867188},
{233,6,0.0014808177947998047},
{234,6,0.0014989376068115234},
{235,6,0.0015347003936767578},
{236,6,0.0015513896942138672},
{237,6,0.0015439987182617188},
{238,6,0.001424551010131836},
{239,6,0.0015001296997070312},
{240,6,0.0014910697937011719},
{241,6,0.0015096664428710938},
{242,6,0.0

{250,7,0.001650094985961914},
{251,7,0.0016448497772216797},
{252,7,0.0016710758209228516},
{253,7,0.0015423297882080078},
{254,7,0.0016694068908691406},
{255,7,0.0016477108001708984},
{256,7,0.0016217231750488281},
{4,8,0.00015497207641601562},
{5,8,0.00011110305786132812},
{6,8,0.00014519691467285156},
{7,8,0.00011587142944335938},
{8,8,0.00014710426330566406},
{9,8,0.0001461505889892578},
{10,8,0.00012612342834472656},
{11,8,0.0001270771026611328},
{12,8,0.0001480579376220703},
{13,8,0.0001456737518310547},
{14,8,0.0001633167266845703},
{15,8,0.0002014636993408203},
{16,8,0.00018024444580078125},
{17,8,0.0001804828643798828},
{18,8,0.00020432472229003906},
{19,8,0.0001430511474609375},
{20,8,0.0001251697540283203},
{21,8,0.00012493133544921875},
{22,8,0.00013375282287597656},
{23,8,0.00019693374633789062},
{24,8,0.0001430511474609375},
{25,8,0.00014281272888183594},
{26,8,0.0001537799835205078},
{27,8,0.0001552104949951172},
{28,8,0.0001628398895263672},
{29,8,0.00016355514526367188

{165,9,0.0011279582977294922},
{166,9,0.0010237693786621094},
{167,9,0.0009772777557373047},
{168,9,0.0010461807250976562},
{169,9,0.0010406970977783203},
{170,9,0.0010623931884765625},
{171,9,0.0010035037994384766},
{172,9,0.0010116100311279297},
{173,9,0.0011017322540283203},
{174,9,0.001088857650756836},
{175,9,0.0011203289031982422},
{176,9,0.0014619827270507812},
{177,9,0.0011806488037109375},
{178,9,0.0010313987731933594},
{179,9,0.0010349750518798828},
{180,9,0.0010411739349365234},
{181,9,0.0010666847229003906},
{182,9,0.0011076927185058594},
{183,9,0.0011277198791503906},
{184,9,0.001154184341430664},
{185,9,0.0011506080627441406},
{186,9,0.0010912418365478516},
{187,9,0.0011227130889892578},
{188,9,0.0011153221130371094},
{189,9,0.0011169910430908203},
{190,9,0.0011539459228515625},
{191,9,0.0011456012725830078},
{192,9,0.0011572837829589844},
{193,9,0.0011551380157470703},
{194,9,0.0011706352233886719},
{195,9,0.0011773109436035156},
{196,9,0.0011708736419677734},
{197,9,0.0

...

{220,248,0.0041348934173583984},
{221,248,0.004161834716796875},
{222,248,0.004208087921142578},
{223,248,0.004115104675292969},
{224,248,0.0041315555572509766},
{225,248,0.004765510559082031},
{226,248,0.004559040069580078},
{227,248,0.0042493343353271484},
{228,248,0.004145383834838867},
{229,248,0.004241228103637695},
{230,248,0.004251956939697266},
{231,248,0.00429534912109375},
{232,248,0.0042765140533447266},
{233,248,0.004366397857666016},
{234,248,0.004391193389892578},
{235,248,0.0042858123779296875},
{236,248,0.0044133663177490234},
{237,248,0.004385471343994141},
{238,248,0.004414796829223633},
{239,248,0.0046482086181640625},
{240,248,0.004447221755981445},
{241,248,0.004498958587646484},
{242,248,0.004593849182128906},
{243,248,0.00469207763671875},
{244,248,0.004630565643310547},
{245,248,0.0045778751373291016},
{246,248,0.004670619964599609},
{247,248,0.004708528518676758},
{248,248,0.0048220157623291016},
{249,248,0.004843950271606445},
{250,248,0.0048906803131103516},


{91,250,0.0035016536712646484},
{92,250,0.004175424575805664},
{93,250,0.0054967403411865234},
{94,250,0.004054069519042969},
{95,250,0.0037136077880859375},
{96,250,0.003793001174926758},
{97,250,0.0037953853607177734},
{98,250,0.003848552703857422},
{99,250,0.003874063491821289},
{100,250,0.0038864612579345703},
{101,250,0.0033872127532958984},
{102,250,0.0034558773040771484},
{103,250,0.00347137451171875},
{104,250,0.003550291061401367},
{105,250,0.0035371780395507812},
{106,250,0.0035877227783203125},
{107,250,0.0036039352416992188},
{108,250,0.003705263137817383},
{109,250,0.0036885738372802734},
{110,250,0.0037560462951660156},
{111,250,0.0037598609924316406},
{112,250,0.0037190914154052734},
{113,250,0.0036144256591796875},
{114,250,0.003785371780395508},
{115,250,0.00401616096496582},
{116,250,0.004026651382446289},
{117,250,0.003942966461181641},
{118,250,0.004120349884033203},
{119,250,0.004026889801025391},
{120,250,0.004151821136474609},
{121,250,0.003531217575073242},
{122

{169,251,0.0039403438568115234},
{170,251,0.003972768783569336},
{171,251,0.004022121429443359},
{172,251,0.004033327102661133},
{173,251,0.0040395259857177734},
{174,251,0.004173755645751953},
{175,251,0.004148960113525391},
{176,251,0.004165172576904297},
{177,251,0.0041387081146240234},
{178,251,0.004171609878540039},
{179,251,0.004164934158325195},
{180,251,0.005127429962158203},
{181,251,0.0054781436920166016},
{182,251,0.004716157913208008},
{183,251,0.004378795623779297},
{184,251,0.004425764083862305},
{185,251,0.0044133663177490234},
{186,251,0.004447460174560547},
{187,251,0.004498481750488281},
{188,251,0.004518747329711914},
{189,251,0.004601240158081055},
{190,251,0.0046002864837646484},
{191,251,0.004598379135131836},
{192,251,0.004630565643310547},
{193,251,0.004682064056396484},
{194,251,0.004704713821411133},
{195,251,0.0046994686126708984},
{196,251,0.004722118377685547},
{197,251,0.004555463790893555},
{198,251,0.004656791687011719},
{199,251,0.004766941070556641},
{

{184,253,0.0043487548828125},
{185,253,0.004275083541870117},
{186,253,0.0047359466552734375},
{187,253,0.005289793014526367},
{188,253,0.005233049392700195},
{189,253,0.004529476165771484},
{190,253,0.004456758499145508},
{191,253,0.004423856735229492},
{192,253,0.004551410675048828},
{193,253,0.004510164260864258},
{194,253,0.004602193832397461},
{195,253,0.004679203033447266},
{196,253,0.0048274993896484375},
{197,253,0.004671335220336914},
{198,253,0.004748106002807617},
{199,253,0.004728078842163086},
{200,253,0.004847049713134766},
{201,253,0.00474095344543457},
{202,253,0.004981279373168945},
{203,253,0.003614664077758789},
{204,253,0.0037584304809570312},
{205,253,0.003805875778198242},
{206,253,0.0037152767181396484},
{207,253,0.0036802291870117188},
{208,253,0.0037190914154052734},
{209,253,0.0039031505584716797},
{210,253,0.003821134567260742},
{211,253,0.0039215087890625},
{212,253,0.0038690567016601562},
{213,253,0.0038900375366210938},
{214,253,0.003937721252441406},
{215

{233,255,0.004178285598754883},
{234,255,0.004225015640258789},
{235,255,0.004252433776855469},
{236,255,0.004364728927612305},
{237,255,0.0042231082916259766},
{238,255,0.005229949951171875},
{239,255,0.005002021789550781},
{240,255,0.004449605941772461},
{241,255,0.004288196563720703},
{242,255,0.004514932632446289},
{243,255,0.004521369934082031},
{244,255,0.004523277282714844},
{245,255,0.004519462585449219},
{246,255,0.004547119140625},
{247,255,0.004567146301269531},
{248,255,0.004468202590942383},
{249,255,0.004475116729736328},
{250,255,0.00458979606628418},
{251,255,0.004570722579956055},
{252,255,0.004614353179931641},
{253,255,0.004672527313232422},
{254,255,0.00461888313293457},
{255,255,0.004666566848754883},
{256,255,0.004611492156982422},
{4,256,0.003419637680053711},
{5,256,0.0027413368225097656},
{6,256,0.0032258033752441406},
{7,256,0.0028111934661865234},
{8,256,0.0032835006713867188},
{9,256,0.0029065608978271484},
{10,256,0.0031888484954833984},
{11,256,0.002980709